In [1]:
import requests
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionUpscalePipeline
import torch

In [ ]:
model_id = "stabilityai/stable-diffusion-x4-upscaler"
pipeline = StableDiffusionUpscalePipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipeline = pipeline.to("cuda")


In [ ]:
url = "https://webeng.kaist.ac.kr/webengpress/wp-content/uploads/2013/04/seunghyun.png"
response = requests.get(url)
low_res_img = Image.open("/workspaces/ModelHubTest/src/composition/samples/0E96XUHZGSOP.jpg")
# low_res_img = low_res_img.resize((128, 128))

In [ ]:
prompt = "a vehicle"

upscaled_image = pipeline(prompt=prompt, image=low_res_img).images[0]
upscaled_image

In [ ]:
from transformers import DetrImageProcessor, DetrForObjectDetection, pipeline
import torch
from PIL import Image, ImageDraw
import requests
import matplotlib.pyplot as plt

In [ ]:
objdet_processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
objdet_model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")


In [ ]:
inputs = objdet_processor(images=upscaled_image, return_tensors="pt")
outputs = objdet_model(**inputs)

In [ ]:
target_sizes = torch.tensor([upscaled_image.size[::-1]])
results = objdet_processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]
draw = ImageDraw.Draw(upscaled_image)
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    label_text = f"{objdet_model.config.id2label[label.item()]}: {round(score.item(), 3)}"
    draw.rectangle(box, outline="red", width=3)
    draw.text((box[0], box[1]), label_text, fill="red")
    print(
            f"Detected {objdet_model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )

plt.figure(figsize=(10, 10))
plt.imshow(upscaled_image)
plt.axis("off")
plt.show()

In [ ]:
from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation




In [ ]:
processor = AutoImageProcessor.from_pretrained("facebook/mask2former-swin-tiny-coco-instance")
model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-tiny-coco-instance")


In [ ]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
inputs = processor(images=image, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

In [ ]:
# model predicts class_queries_logits of shape `(batch_size, num_queries)`
# and masks_queries_logits of shape `(batch_size, num_queries, height, width)`
class_queries_logits = outputs.class_queries_logits
masks_queries_logits = outputs.masks_queries_logits

In [ ]:
# you can pass them to processor for postprocessing
result = processor.post_process_instance_segmentation(outputs, target_sizes=[image.size[::-1]])[0]
# we refer to the demo notebooks for visualization (see "Resources" section in the Mask2Former docs)
predicted_instance_map = result["segmentation"]
predicted_instance_map

In [ ]:
classifier_model = "tigeryi/imagenet-tiger"

classifier = pipeline(model=classifier_model, device=0)




In [ ]:
image = "/workspaces/ModelHubTest/src/composition/samples/th-2550814735.jpg"
classifier(image)

In [ ]:
from urllib.request import urlopen

img = Image.open(urlopen(
    'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'
))

img

In [ ]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

image

In [ ]:
#timm
import timm

model = timm.create_model("edadaltocg/resnet18_cifar100", pretrained=True)
model = model.eval()

data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

output = model(transforms(img).unsqueeze(0))  # unsqueeze single image into batch of 1

top1_probabilities, top1_class_indices = torch.topk(output.softmax(dim=1) * 100, k=1)

top1_class_indices

In [8]:
from transformers import DetrImageProcessor, DetrForObjectDetection, pipeline
import numpy as np
import matplotlib.pyplot as plt
import torch
import requests
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionUpscalePipeline
import torch

In [3]:
def upscaling(image, prompt="a vehicle"):
    model_id = "stabilityai/stable-diffusion-x4-upscaler"
    pipeline = StableDiffusionUpscalePipeline.from_pretrained(
        model_id, torch_dtype=torch.float16
    )
    pipeline = pipeline.to("cuda")

    upscaled_image = pipeline(prompt=prompt, image=image).images[0]
    return upscaled_image

def segmentation(image, relevant_labels):
    processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-101", revision="no_timm")
    model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-101", revision="no_timm")

    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)

    # Convert outputs (bounding boxes and class logits) to COCO API
    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.6)[0]
    print(f"Detection Results: {results}")

    detected_patches = []

    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        box = [round(i, 2) for i in box.tolist()]
        if str(label.item()) in relevant_labels:
            print(
                f"Detected {model.config.id2label[label.item()]} with confidence "
                f"{round(score.item(), 3)} at location {box}"
            )
            # Extract the sub-patch from the image
            left, top, right, bottom = box
            sub_patch = image.crop((left, top, right, bottom))
            detected_patches.append((sub_patch, score.item(), label.item(), box))


    return detected_patches


def classification(image):
    model_id = "apple/mobilevit-small"
    classifier = pipeline(model=model_id, device=0)
    return classifier(image)


In [ ]:
def composition_runner(sampled_images):
    relevant_labels = {"2": "bicycle", "3": "car", "6": "bus", "7": "train", "8": "truck"}
    predictions = []
    for image in sampled_images:
        u_i = upscaling(image)
        d_p = segmentation(u_i, relevant_labels)
        for patch, score, label, box in d_p:
            predictions.append(classification(patch))
    
    return predictions

def image_sampler(dataset):
    pass

In [10]:
def visualize_patches(detected_patches):
    plt.figure(figsize=(12, 8))
    for i, (patch, score, label, box) in enumerate(detected_patches):
        plt.subplot(1, len(detected_patches), i + 1)
        plt.imshow(patch)
        plt.title(f"Label: {label}, Score: {round(score, 2)}")
        plt.axis('off')
    plt.show()

In [ ]:
from datasets import load_dataset

def create_dataset(filepath, n_samples=5849):
    dataset = load_dataset(
        "imagefolder",
        data_dir=filepath,
        split="test",
    )
    # dataset = dataset.shuffle(seed=42).select(range(n_samples))
    return dataset

In [ ]:
vehicle_dataset = create_dataset("/workspaces/ModelHubTest/src/dataset/vehicles")

In [12]:
img = Image.open("/workspaces/ModelHubTest/src/composition/samples/24CCSBI3QYFH.jpg")
upscaled_image = upscaling(img)

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

In [13]:
upscaled_image.save("/workspaces/ModelHubTest/src/composition/outputs/upscaled.png")

In [14]:
relevant_labels = {"2": "bicycle", "3": "car", "6": "bus", "7": "train", "8": "truck"}
detected_patches = segmentation(upscaled_image, relevant_labels)
visualize_patches(detected_patches)

Detection Results: {'scores': tensor([0.9994, 0.6095, 0.6447, 0.9385], grad_fn=<IndexBackward0>), 'labels': tensor([1, 4, 4, 4]), 'boxes': tensor([[ 3.2810e+02, -2.2661e-01,  4.8027e+02,  3.9414e+02],
        [ 1.5074e+01,  2.1815e+02,  4.8614e+02,  5.0512e+02],
        [ 1.4051e+01,  1.3868e+02,  4.8639e+02,  4.4360e+02],
        [ 1.4478e+01,  1.3694e+02,  4.8621e+02,  5.0943e+02]],
       grad_fn=<IndexBackward0>)}


<Figure size 1200x800 with 0 Axes>

In [ ]:
for patch, score, label, box in detected_patches:
    print(classification(patch))

In [ ]:
print(composition_runner(vehicle_dataset[:5]["image"]))

In [3]:
import pandas as pd

ModuleNotFoundError: No module named 'huggingface_model_server'

In [ ]:
class CLF_Models:
    def __init__(self):
        self.models = pd.DataFrame()
        self.inference_models = None
        self.test_models = None
    
    def __str__(self):
        return (
            f"All Classification Models:\n{self.models['model']},\n"
            f"Inference Models:\n{self.inference_models['model']},\n"
            f"Test Models:\n{self.test_models['model']},\n"
        )
    
    def get_models_from_csv(self, filepath):
        self.models = pd.read_csv(filepath)
    
    def random_sample_inference_models(self, count):
        self.inference_models = self.models.sample(count)
        self.test_models = self.models.drop(self.inference_models.index)

clf_models = CLF_Models()
clf_models.get_models_from_csv("/workspaces/ModelHubTest/src/composition/composition_classification_models.csv")
clf_models.random_sample_inference_models(10)
print(clf_models)


In [1]:
test = [[[{'label': 'tractor', 'score': 0.38368913531303406}, {'label': 'thresher, thrasher, threshing machine', 'score': 0.3412572741508484}, {'label': 'harvester, reaper', 'score': 0.07035797089338303}, {'label': 'bulletproof vest', 'score': 0.033604104071855545}, {'label': 'jean, blue jean, denim', 'score': 0.018526067957282066}], [{'label': 'thresher, thrasher, threshing machine', 'score': 0.48355886340141296}, {'label': 'tractor', 'score': 0.38515520095825195}, {'label': 'harvester, reaper', 'score': 0.018105223774909973}, {'label': 'tow truck, tow car, wrecker', 'score': 0.015623060055077076}, {'label': 'trailer truck, tractor trailer, trucking rig, rig, articulated lorry, semi', 'score': 0.007327983621507883}], [{'label': 'passenger car, coach, carriage', 'score': 0.46890807151794434}, {'label': 'planetarium', 'score': 0.18813657760620117}, {'label': 'streetcar, tram, tramcar, trolley, trolley car', 'score': 0.05928593873977661}, {'label': 'bullet train, bullet', 'score': 0.03684035688638687}, {'label': 'prison, prison house', 'score': 0.029240326955914497}]]]
test[0]

[[{'label': 'tractor', 'score': 0.38368913531303406},
  {'label': 'thresher, thrasher, threshing machine',
   'score': 0.3412572741508484},
  {'label': 'harvester, reaper', 'score': 0.07035797089338303},
  {'label': 'bulletproof vest', 'score': 0.033604104071855545},
  {'label': 'jean, blue jean, denim', 'score': 0.018526067957282066}],
 [{'label': 'thresher, thrasher, threshing machine',
   'score': 0.48355886340141296},
  {'label': 'tractor', 'score': 0.38515520095825195},
  {'label': 'harvester, reaper', 'score': 0.018105223774909973},
  {'label': 'tow truck, tow car, wrecker', 'score': 0.015623060055077076},
  {'label': 'trailer truck, tractor trailer, trucking rig, rig, articulated lorry, semi',
   'score': 0.007327983621507883}],
 [{'label': 'passenger car, coach, carriage', 'score': 0.46890807151794434},
  {'label': 'planetarium', 'score': 0.18813657760620117},
  {'label': 'streetcar, tram, tramcar, trolley, trolley car',
   'score': 0.05928593873977661},
  {'label': 'bullet tra